In [20]:
# Librairies
import pandas as pd
import numpy as np
from pickleshare import PickleShareDB

from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [21]:
# Pour garantir la reproductibilité
np.random.seed(42)  # Pour numpy

## Chargement des données
Nous chargeons les données depuis le fichier des données préprarées.

In [22]:
# Données nettoyées
db = PickleShareDB('../prep_data/kity')

# df_net_1 = db['net_attack_1_clean']
# df_net_2 = db['net_attack_2_clean']
# df_net_3 = db['net_attack_3_clean']
# df_net_4 = db['net_attack_4_clean']
# df_net_norm = db['net_norm_clean']

In [36]:
# Création d'un dataset avec toutes les données
df = [db['net_attack_1_clean'], db['net_attack_2_clean'], db['net_attack_3_clean'], db['net_attack_4_clean']]
df = pd.concat(df, axis=0, ignore_index=True)

In [37]:
# Stratified sampling: maintain class distribution
df = df.groupby('label', group_keys=False).apply(
    lambda x: x.sample(frac=0.02, random_state=42)
)

/tmp/ipykernel_9592/3121293707.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('label', group_keys=False).apply(


In [38]:
df['label'].value_counts()

label
normal            253920
DoS               103231
MITM               43108
physical fault     30970
anomaly                8
scan                   1
Name: count, dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 431238 entries, 21526548 to 5737630
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Time             431238 non-null  datetime64[ns]
 1   mac_s            431238 non-null  object        
 2   mac_d            431238 non-null  object        
 3   ip_s             431238 non-null  object        
 4   ip_d             431238 non-null  object        
 5   sport            431238 non-null  category      
 6   dport            431238 non-null  category      
 7   proto            431238 non-null  object        
 8   flags            431238 non-null  object        
 9   size             431238 non-null  int64         
 10  modbus_fn        431238 non-null  object        
 11  n_pkt_src        431238 non-null  Int64         
 12  n_pkt_dst        431238 non-null  Int64         
 13  modbus_response  431238 non-null  object        
 14  label_n          

## Preparation pour les modeles

La colonne Time, au format datetime, n'étant pas directement interprétable pour la plupart des modeles, nous la supprimons.

In [40]:
df = df.drop(columns='Time')

In [41]:
unique_count = df.nunique()
print(f" Number of unique values :\n{unique_count}")

 Number of unique values :
mac_s                 9
mac_d                10
ip_s                  9
ip_d                  9
sport                 4
dport                 4
proto                 5
flags                 9
size                 23
modbus_fn             5
n_pkt_src           101
n_pkt_dst           101
modbus_response    2259
label_n               2
label                 6
is_duplicate          2
dtype: int64


On a les colonnes de types objet, categorie et numerique

Pour les colonnes catégorielles avec peu de valeurs différentes, nous avons réalisé un One-Hot afin de transformer leurs catégories en variables binaires.  
Cela permet de traiter correctement ces colonnes non numériques, en concervant les valeurs des catégories.

In [42]:
df = pd.get_dummies(df, columns=['flags', 'dport', 'sport', 'proto', 'modbus_fn'], prefix=['flags', 'dport', 'sport', 'proto', 'modbus_fn'])

Pour les colonnes de type object, nous avons appliqué un encodage ordinal pour convertir les catégories en valeurs numériques, afin qu'elles soient exploitables.
Puis nous les standardisons pour qu'elles aient le même poids que les autres colonnes numériques.

In [43]:
ordinal_encoder = OrdinalEncoder()
df[['ip_s', 'ip_d', 'mac_s', 'mac_d']] = ordinal_encoder.fit_transform(df[['ip_s', 'ip_d', 'mac_s', 'mac_d']])
scaler = StandardScaler()
df[['ip_s', 'ip_d', 'mac_s', 'mac_d']] = scaler.fit_transform(df[['ip_s', 'ip_d', 'mac_s', 'mac_d']])

Pour la colonne modbus_response, compte tenu du grand nombre de valeurs distinctes (2000+), nous avons adopté une approche différente. Nous avons conservé les catégories représentant au moins 0,1 % des données, en regroupant les catégories moins fréquentes sous une nouvelle catégorie intitulée "other".  Enfin, nous avons appliqué un One-Hot au résultat pour représenter ces catégories de manière explicite, comme pour les autres colonnes catégoriques.

In [44]:
df['modbus_response'] = df['modbus_response'].str.replace(r'\[|\]', '', regex=True)
threshold = 0.001 * len(df)
value_counts = df['modbus_response'].value_counts()
frequent_categories = value_counts[value_counts > threshold].index
df['modbus_response'] = np.where(df['modbus_response'].isin(frequent_categories), df['modbus_response'], 'autre')
df = pd.get_dummies(df, columns=['modbus_response'], prefix=['modbus_response'])

In [32]:
# df.drop(columns='modbus_response', inplace=True)

Nous supprimons les colonnes créée à cause des valeurs manquantes, car elles n'ont pas d'utilitées.

In [45]:
df.drop(columns=['modbus_fn_inconnue', 'flags_inconnue', 'sport_inconnu', 'dport_inconnu', 'modbus_response_inconnue'], inplace=True)

Nous normalisons les colonnes au format numérique.

In [46]:
scaler = StandardScaler()
df[['n_pkt_src', 'n_pkt_dst', 'size']] = scaler.fit_transform(df[['n_pkt_src', 'n_pkt_dst', 'size']])

Nous transformons les valeurs booléennes en float afin qu'elles soient traitées correctement.

In [47]:
for col in df.columns:
    if df[col].dtype == 'bool' and col != 'label_n':
        df[col] = df[col].astype(float)

### Vérification

In [48]:
df.head()

,mac_s,mac_d,ip_s,ip_d,size,n_pkt_src,n_pkt_dst,label_n,label,is_duplicate,...,modbus_fn_Read Coils Response,modbus_fn_Read Holding Registers,modbus_fn_Read Holding Registers Response,modbus_response_0,modbus_response_1,modbus_response_11,modbus_response_4000,modbus_response_450,modbus_response_458,modbus_response_autre
21526548,0.419734,1.525168,0.936518,-0.922720,-0.381005,0.394144,-0.992415,True,DoS,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13106703,-1.463958,-1.401431,0.216413,0.518142,2.613811,1.820428,1.870674,True,DoS,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17889198,-1.934881,-1.401431,-0.863744,0.518142,-0.393415,1.496273,1.475765,True,DoS,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18693138,-1.934881,-1.401431,0.936518,0.518142,-0.393415,1.723181,1.673219,True,DoS,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11368320,-1.463958,0.354529,0.936518,0.878357,-0.393415,1.399026,1.442856,True,DoS,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Sampling
On va utiliser un subset de taille 500 000 pour l'entrainement.
Ca permet de garder une partie sinificative des donnees (environ 10%), sans que le dataset soit considere comme large et devient cher au niveau de calcul.

In [ ]:
# sample_df = df.sample(n=500000)

## Enregistrement dans PickleShare

In [49]:
db['net_sample_for_models'] = df